In [1]:
"""
直接去电商宝复制请求标头
然后开始自动采集订单
"""

import re

import pyperclip
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from requests_html import HTMLSession

session = HTMLSession()

# 构建数据库
uri = "mongodb://192.168.0.110:27017/"
client = MongoClient(uri, server_api=ServerApi("1"))
database = client["电商宝返款订单"]
collect = database["订单列表"]

# 获取请求标头
req_header = pyperclip.paste().strip()

# 构建所有url列表
req_split = req_header.split("\n")
url: str = "https://appscrm.ecbao.cn" + re.findall(r"\s(.*?)\s", req_split[0])[0]
all_url = []
for x in range(1, 10000):
    all_url.append(url.replace("page=1", f"page={x}"))

# 构建请求标头
header_dict = {}
for req_line in req_split[1:]:
    req_line = req_line.strip()
    line_split = req_line.split(": ")
    header_dict.update({line_split[0]: line_split[1]})

for num, url in enumerate(all_url):
    print(f"开始采集第{num+1}页")

    with session.post(url, headers=header_dict) as res:
        objs = res.json().get("data").get("data")

        for obj in objs:
            order_sn = obj.get("order_sn")
            count = collect.count_documents(filter={"order_sn": obj.get("order_sn")})
            if count == 0:
                insert_res = collect.insert_one(obj)
                print(
                    f'插入数据：{insert_res.inserted_id}\nwechat:{obj.get("wechat_nick")}'
                )

            else:
                print("数据存在")
                break

    break

开始采集第1页
插入数据：671364949dbc769e608ea268
wechat:°
插入数据：671364949dbc769e608ea269
wechat:微信用户
插入数据：671364949dbc769e608ea26a
wechat:我是小毛
插入数据：671364949dbc769e608ea26b
wechat:Judy
插入数据：671364959dbc769e608ea26c
wechat:。
插入数据：671364959dbc769e608ea26d
wechat:蕾 Flora
插入数据：671364959dbc769e608ea26e
wechat:琦美君
数据存在
